In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('demographics.csv')

In [3]:
df['age'] = df['age'].where(df['age'] <= 89, 89)

In [4]:
def normalise_eth(eth) :
    eth = eth.split('-')[0].strip().split('/')[0].strip().split(' OR ')[0].strip()
    if eth in ['PATIENT DECLINED TO ANSWER', 'UNABLE TO OBTAIN', 'UNKNOWN', 'OTHER'] :
        eth = 'OTHER'
        
    return eth

In [5]:
df['ethnicity_combined'] = df['ethnicity'].apply(
    lambda x : normalise_eth(x)
)

In [6]:
map_gender = { "F" : 0, "M" : 1 }

In [7]:
df['gender_y'] = df['gender'].apply(lambda s : map_gender[s])

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
eth_encoder = LabelEncoder()
eth_encoder.fit(df['ethnicity_combined'])

LabelEncoder()

In [10]:
import pickle
pickle.dump(eth_encoder, open('eth_encoder.p', 'wb'))

In [11]:
df['ethnicity_y'] = eth_encoder.transform(df['ethnicity_combined'])

In [12]:
from sklearn.preprocessing import KBinsDiscretizer

In [13]:
kbage = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
kbage.fit(np.array(df['age']).reshape(-1, 1))

KBinsDiscretizer(encode='ordinal', n_bins=4, strategy='quantile')

In [14]:
pickle.dump(kbage, open('age_encoder.p', 'wb'))

In [15]:
df['age_y'] = kbage.transform(np.array(df['age']).reshape(-1, 1)).astype(np.int)

In [16]:
df.to_csv("normalised_demographics.csv", index=False)

In [17]:
", ".join([x[1].title() + ":" + str(x[0]) for x in list(enumerate(eth_encoder.classes_))])

'American Indian:0, Asian:1, Black:2, Caribbean Island:3, Hispanic:4, Middle Eastern:5, Multi Race Ethnicity:6, Native Hawaiian:7, Other:8, Portuguese:9, South American:10, White:11'